In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

noisy_img_arr = cv2.imread('./images/binary_gaussnoise.jpg',0)
img_arr = cv2.imread('./images/binary_image.jpg',0)
# print('test shape: ', noisy_img_arr.shape)
# cv2.imshow('test', noisy_img_arr)

def percent_pixel_flipped(noisy_arr,denoised_arr):
    num_flipped = 0
    total_px = noisy_arr.shape[0]*noisy_arr.shape[1]
    for i in range(noisy_arr.shape[0]):
        for j in range(noisy_arr.shape[1]):
            if noisy_arr[i,j]!= denoised_arr[i,j]:
                num_flipped+=1
   
    percent_change = num_flipped*100./total_px
    return percent_change

def check_limit(value, limit):
    if value<0:
        value=limit-1
    if value==limit:
        value=0
    return value

def energyfx(y_array, x_array, xval,yval, constant):
    h = constant[0]
    b = constant[1]
    n = constant[2]
    total_pixels = x_array.shape[0]*x_array.shape[1]
    energy = h * x_array[xval, yval] 
    energy += - n * x_array[xval, yval] * y_array[xval, yval]
    x_neighbour = [-1,1]
    y_neighbour = [-1,1]
    for i in x_neighbour:
        for j in y_neighbour:
            x_n = check_limit(xval +i,x_array.shape[0])
            y_n = check_limit(yval +j, x_array.shape[1])
            
            energy += -b*x_array[xval,yval]*x_array[x_n,y_n]
    energy = energy/total_pixels
    return energy

def calculate_total_energy(y_array,x_array,constant):
    energy = 0.
    for i in range(y_array.shape[0]):
        for j in range(y_array.shape[1]):
            energy += energyfx(y_array, x_array, i,j, constant)
    return energy

#this list is [h, beta,eta]
constant = [0,.1,.02]
hidden_image = np.copy(noisy_img_arr)
total_energy= calculate_total_energy(noisy_img_arr, hidden_image, constant)
print('Total Energy: ', total_energy)

def icm_single_pixel(visible_arr, hidden_arr, px_x, px_y, total_energy, const_list):
    current_energy = energyfx(visible_arr, hidden_arr,px_x,px_y, const_list)
    other_energy = total_energy - current_energy
    #flip the pixel
    new_hidden_arr = np.copy(hidden_arr)
    if hidden_arr[px_x,px_y]==1:
        new_hidden_arr[px_x,px_y]=-1
    else:
        new_hidden_arr[px_x,px_y] = 1
    flipped_energy = energyfx(visible_arr, new_hidden_arr,px_x,px_y, const_list)
    #print current_energy, flipped_energy
    if flipped_energy < current_energy:
        should_flip = True
        total_energy = other_energy + flipped_energy
        hidden_arr = new_hidden_arr
        #print percent_pixel_flipped(hidden_arr, visible_arr)
    else:
        should_flip = False
    
    return (hidden_arr,should_flip,total_energy)
    #return (should_flip, hidden_arr, total_energy)

#main icm simulation
hidden_image = np.copy(noisy_img_arr)
energy_this_round = total_energy
print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))

for sim_round in range(10):
    for i in range(hidden_image.shape[0]):
        for j in range(hidden_image.shape[1]):
            hidden_image,should_flip,total_energy = icm_single_pixel(noisy_img_arr,hidden_image,i,j, total_energy,constant)
            
        #print percent_pixel_flipped(hidden_image, lena_arr)
    if (total_energy - energy_this_round) == 0:
        print ("Algorithm converged")
        break
    energy_this_round = total_energy
    print ("Total Energy:",total_energy)
    print ("% Pixels flipped:", percent_pixel_flipped(hidden_image, img_arr))

cv2.imshow('latent variables', hidden_image)
# print(hidden_image)
print(percent_pixel_flipped(hidden_image, img_arr))
cv2.waitKey(0)
cv2.destroyAllWindows()

Total Energy:  -9617.131070150144
% Pixels flipped: 56.67363530778165
Total Energy: -10526.17399256759
% Pixels flipped: 73.51219512195122
Total Energy: -15779.35805529013
% Pixels flipped: 69.335656213705
Total Energy: -15780.561678983508
% Pixels flipped: 69.335656213705
Algorithm converged
69.335656213705
